# __오디오 파일을 받아쓰는 음성 인식 모델 사용하기__

- 튜토리얼 난이도: ★☆☆☆☆
- 읽는데 걸리는 시간: 5분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_ml/audio_recognition/speech_recognition2.ipynb
- 참고 문서: [(AI-Hub) 한국어음성 데이터](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=123), [whisper](https://github.com/openai/whisper)

## 튜토리얼 소개 

<div class="admonition note">
    <h4 class="admonition-title">음성 인식 기술 이해하기</h4>
    <p>컴퓨터 음성 인식 또는 음성-텍스트 변환(Speech-to-Text)이라고도 부르는 음성 인식 기술은 프로그램이 사람의 음성을 텍스트 형식으로 처리할 수 있도록 해줍니다. 최근 자동차, 의료 분야, 인공지능 스피커나 스마트폰을 이용한 일상 생활까지 광범위한 분야에서 활용되고 있습니다. 최근 <a href="https://ko.wikipedia.org/wiki/%EA%B8%B0%EA%B3%84_%ED%95%99%EC%8A%B5">머신러닝(기계학습/Machine Learning)</a> 알고리즘을 활용한 음성 인식 기술은 문법, 구문, 구조, 오디오와 음성 신호의 구성을 통합하여 음성을 이해하고 처리합니다.</p>
</div>

<div class="admonition warning">
    <p>일반적으로 음성 인식(Voice Recognition)과 혼동 될 수 있는데, 음성 인식은 개별 사용자의 목소리를 식별하는 데만 집중합니다.</p>
</div>

오늘날 음성 인식 기술은 다양한 산업 분야에서 응용되고 있습니다. 음성 인식 기술의 발전은 단순 여행용 자동 통역에서 난이도가 높은 비즈니스 회의 통역까지 확대되고 있습니다. 또한, 음성 인식 기술은 음성 합성 기술로 발전하여 가상 안내원이나 비서 역할을 수행하기도 하고 특정 연예인이나 유명인의 목소리를 흉내내어 정해진 지문을 목소리로 변환하기도 하며 응용되고 있습니다.

__아래는 ThanoSQL 음성 인식 모델의 활용 및 예시입니다.__

- 음성 인식 기술은 전화 상담 데이터를 텍스트로 변환하여 고객의 감정 분석이나 상담 트렌드 분석 등을 가능하게 합니다. 상담자는 음성 인식 기술을 통해, 고객의 문의에 대한 답변이 되는 정보나 과거의 유사 사례 등을 빠르게 제공받아 고객 상담을 개선할 수 있습니다.
또한, 상담 종료 후에는 음성으로 저장되어 있는 데이터를 활용하여 감정 분석을 통해 고객의 만족도 등을 간접적으로 측정하여 트렌드 별 고객의 만족도를 확인할 수 있습니다.
- 음성 인식 기술을 사용하면 키보드로 작성하는 메모보다 빠르게 작성이 가능하고, 긴 음성 파일에서도 빠르게 특정 키워드를 검색 할 수 있습니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 <strong>Whisper</strong> [Alec Radford et al. 2022]는 OpenAI에서 공개한 범용 음성 인식 딥러닝 모델로, 다양한 오디오의 대규모 데이터 세트에 대한 학습을 지원하며, 다국어 음성 인식은 물론 음성 번역 및 언어 식별을 수행할 수 있게 하는 다중 작업 모델입니다. 일반적인 음성 인식 문제에 적용해도 좋은 성능을 보입니다. 본 튜토리얼에서는 <strong>Whisper</strong>의 기능 중 일반적으로 사용되는 음성 인식과 영어로 번역을 실행합니다.</p>
</div>

## __0. 데이터 세트 및 모델 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET korean_voice_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY korean_voice
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/korean_voice_data/korean_voice.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

### __모델 준비__

In [4]:
%%thanosql
GET THANOSQL MODEL whisper_s
OPTIONS (
    model_name='tutorial_whisper_small',
    overwrite=True
    )

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL MODEL</strong>" 쿼리 구문을 사용하여 원하는 모델을 워크스페이스 및 데이터베이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL MODEL</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>“model_name”: 저장할 모델의 이름 (str, optional)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

본 튜토리얼을 진행하기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __korean_voice__ 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM korean_voice
LIMIT 5

,audio_path,sampling_rate,transcript_phonetic,transcript_spelling,duration
0,thanosql-dataset/korean_voice_data/audio/broad...,16000,가를 보면 한국어 사용하는 인구 수가 십이 위입니다. 일위가 중국어고 이위가 스페인...,가를 보면 한국어 사용하는 인구 수가 십이 위입니다. 일위가 중국어고 이위가 스페인...,8.70
1,thanosql-dataset/korean_voice_data/audio/broad...,16000,말을 사랑하고 아껴서 규정에 맞게 파괴하지 않고 네 잘.,말을 사랑하고 아껴서 규정에 맞게 파괴하지 않고 네 잘.,5.89
2,thanosql-dataset/korean_voice_data/audio/broad...,16000,진행하고 있습니다. 자 오늘의 목표 확인해 보도록 하겠습니다. 오늘의 목표 네.,진행하고 있습니다. 자 오늘의 목표 확인해 보도록 하겠습니다. 오늘의 목표 네.,4.86
3,thanosql-dataset/korean_voice_data/audio/broad...,16000,그리고 이번에는 다른 친구의 글을 평가해보는 것을 하는 겁니다.,그리고 이번에는 다른 친구의 글을 평가해보는 것을 하는 겁니다.,4.61
4,thanosql-dataset/korean_voice_data/audio/broad...,16000,쓰기가 된 글 완성된 글 또는 쓰기 전의 개요 뭐 자료 이런 것들을 보여주면서 그것...,쓰기가 된 글 완성된 글 또는 쓰기 전의 개요 뭐 자료 이런 것들을 보여주면서 그것...,11.52


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>korean_voice</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>audio_path: 음성 파일의 위치 경로</li>
        <li>transcript_spelling: 해당 음성의 목푯값(Target, 스크립트)</li>
        <li>transcript_phonetic: 음성을 기호로 시각적으로 표현한 값</li>
    </ul>
</div>

In [6]:
%%thanosql
PRINT AUDIO
AS
SELECT audio_path
FROM korean_voice
LIMIT 3

/home/jovyan/thanosql-dataset/korean_voice_data/audio/broadcast_00033030.flac


/home/jovyan/thanosql-dataset/korean_voice_data/audio/broadcast_00033057.flac


/home/jovyan/thanosql-dataset/korean_voice_data/audio/broadcast_00033066.flac


## __2. 사전 학습된 모델을 사용하여 음성 인식 결과 예측__

다음 쿼리 구문을 실행하여 __tutorial_whisper_small__ 모델을 사용하여 결과를 예측합니다.

- `task='transcribe'` 옵션을 지정하면 음성 인식을 합니다.

In [7]:
%%thanosql
PREDICT USING tutorial_whisper_small
OPTIONS (
    audio_col='audio_path',
    language='ko',
    task='transcribe',
    result_col='predict_result'
    )
AS
SELECT *
FROM korean_voice

,audio_path,sampling_rate,transcript_phonetic,transcript_spelling,duration,predict_result
0,thanosql-dataset/korean_voice_data/audio/broad...,16000,가를 보면 한국어 사용하는 인구 수가 십이 위입니다. 일위가 중국어고 이위가 스페인...,가를 보면 한국어 사용하는 인구 수가 십이 위입니다. 일위가 중국어고 이위가 스페인...,8.70,가를 보면 한국어 사용하는 인구수가 12위입니다 1위가 중국어고 2위가 스페인어고 ...
1,thanosql-dataset/korean_voice_data/audio/broad...,16000,말을 사랑하고 아껴서 규정에 맞게 파괴하지 않고 네 잘.,말을 사랑하고 아껴서 규정에 맞게 파괴하지 않고 네 잘.,5.89,를 사랑하고 아껴서 규정에 맞게 파괴하지 않고 잘
2,thanosql-dataset/korean_voice_data/audio/broad...,16000,진행하고 있습니다. 자 오늘의 목표 확인해 보도록 하겠습니다. 오늘의 목표 네.,진행하고 있습니다. 자 오늘의 목표 확인해 보도록 하겠습니다. 오늘의 목표 네.,4.86,오늘의 목표 확인해보도록 하겠습니다.
3,thanosql-dataset/korean_voice_data/audio/broad...,16000,그리고 이번에는 다른 친구의 글을 평가해보는 것을 하는 겁니다.,그리고 이번에는 다른 친구의 글을 평가해보는 것을 하는 겁니다.,4.61,그리고 이번에는 다른 친구에게를 평가해 보는 것을 하는 겁니다
4,thanosql-dataset/korean_voice_data/audio/broad...,16000,쓰기가 된 글 완성된 글 또는 쓰기 전의 개요 뭐 자료 이런 것들을 보여주면서 그것...,쓰기가 된 글 완성된 글 또는 쓰기 전의 개요 뭐 자료 이런 것들을 보여주면서 그것...,11.52,"쓰기가 될 글 완성된 글 또는 쓰기 전에 개요, 자료 이런 것들을 보여주면서 그것을..."
...,...,...,...,...,...,...
95,thanosql-dataset/korean_voice_data/audio/broad...,16000,희곡 같은 데서 제일 중요한 한 단어는 뭐라고요.,희곡 같은 데서 제일 중요한 한 단어는 뭐라고요.,3.20,키곡 같은 데에서 제일 중요한 한 단어는 뭐라고요?
96,thanosql-dataset/korean_voice_data/audio/broad...,16000,수필이라는 이름 자체가 무슨 뜻인지 아나요.,수필이라는 이름 자체가 무슨 뜻인지 아나요.,2.94,수필이라는 이름 자체가 무슨 뜻인지 알아요?
97,thanosql-dataset/korean_voice_data/audio/broad...,16000,당근 씨를 막 뿌리려는 남편에게 나는 몇 번이나 말했다 그랬습니다.,당근 씨를 막 뿌리려는 남편에게 나는 몇 번이나 말했다 그랬습니다.,3.58,당근실을 막 뿌리려는 남편에게 나는 몇 번이나 말했다.
98,thanosql-dataset/korean_voice_data/audio/broad...,16000,작년에도 너무 얕게 씨를 뿌려 낭패를 본 적이 있기 때문이다.,작년에도 너무 얕게 씨를 뿌려 낭패를 본 적이 있기 때문이다.,4.22,작년에도 너무 얕게 씨를 뿌려 낭패를 본 적이 있기 때문이다.


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" 쿼리 구문을 사용하여 <strong>tutorial_whisper_small</strong> 모델을 예측에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 예측에 사용할 옵션을 지정합니다.
        <ul>
            <li>"audio_col": 예측에 사용할 오디오의 경로가 기록되어 있는 컬럼의 이름 (str, default: 'audio_path')</li>
            <li>"batch_size": 한 번의 학습에 사용하는 데이터 묶음 속의 데이터 수 (int, optional, default: 16)</li>
            <li>"language": 오디오 파일의 주요 사용 언어를 지정. ‘auto’일 경우 모델이 인식할 수 있는 99가지 언어 중 하나로 인식 (str, default: 'auto')</li>
            <li>"task": 실행할 작업의 종류 (str, 'transcribe'|'translate', default: 'transcribe')</li>
            <li>"result_col": 데이터 테이블에서 예측 결과를 담을 컬럼 이름 (str, optional, default: 'predict_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 사전 학습된 모델을 사용하여 영어로 자동 번역__

다음 쿼리 구문을 실행하여 __tutorial_whisper_small__ 모델을 사용하여 결과를 예측합니다.

- `task='translate'` 옵션을 지정하면 인식된 음성을 영어로 출력합니다. 이 과정은 '한국어 음성'을 바로 '영어 텍스트'로 번역하는 것으로, 중간에 '한국어 텍스트'를 거치지 않는 다는 점이 일반적인 번역 태스크와 다릅니다.

In [8]:
%%thanosql
PREDICT USING tutorial_whisper_small
OPTIONS (
    audio_col='audio_path',
    language='ko',
    task='translate',
    result_col='predict_result'
    )
AS
SELECT *
FROM korean_voice

,audio_path,sampling_rate,transcript_phonetic,transcript_spelling,duration,predict_result
0,thanosql-dataset/korean_voice_data/audio/broad...,16000,가를 보면 한국어 사용하는 인구 수가 십이 위입니다. 일위가 중국어고 이위가 스페인...,가를 보면 한국어 사용하는 인구 수가 십이 위입니다. 일위가 중국어고 이위가 스페인...,8.70,The number of people using Korean is 12.
1,thanosql-dataset/korean_voice_data/audio/broad...,16000,말을 사랑하고 아껴서 규정에 맞게 파괴하지 않고 네 잘.,말을 사랑하고 아껴서 규정에 맞게 파괴하지 않고 네 잘.,5.89,Love and cherish the words and don't destroy t...
2,thanosql-dataset/korean_voice_data/audio/broad...,16000,진행하고 있습니다. 자 오늘의 목표 확인해 보도록 하겠습니다. 오늘의 목표 네.,진행하고 있습니다. 자 오늘의 목표 확인해 보도록 하겠습니다. 오늘의 목표 네.,4.86,Let's check today's goal.
3,thanosql-dataset/korean_voice_data/audio/broad...,16000,그리고 이번에는 다른 친구의 글을 평가해보는 것을 하는 겁니다.,그리고 이번에는 다른 친구의 글을 평가해보는 것을 하는 겁니다.,4.61,"And this time, I'm going to evaluate other fri..."
4,thanosql-dataset/korean_voice_data/audio/broad...,16000,쓰기가 된 글 완성된 글 또는 쓰기 전의 개요 뭐 자료 이런 것들을 보여주면서 그것...,쓰기가 된 글 완성된 글 또는 쓰기 전의 개요 뭐 자료 이런 것들을 보여주면서 그것...,11.52,It is a problem of the order and writing area.
...,...,...,...,...,...,...
95,thanosql-dataset/korean_voice_data/audio/broad...,16000,당근 씨를 막 뿌리려는 남편에게 나는 몇 번이나 말했다 그랬습니다.,당근 씨를 막 뿌리려는 남편에게 나는 몇 번이나 말했다 그랬습니다.,3.58,I told my husband that I would pour carrots a ...
96,thanosql-dataset/korean_voice_data/audio/broad...,16000,작년에도 너무 얕게 씨를 뿌려 낭패를 본 적이 있기 때문이다.,작년에도 너무 얕게 씨를 뿌려 낭패를 본 적이 있기 때문이다.,4.22,I've seen a lot of people who put too little s...
97,thanosql-dataset/korean_voice_data/audio/broad...,16000,하나는 새를 위해서 하나는 또.,하나는 새를 위해서 하나는 또.,2.69,One is for the new year. Another is for the ne...
98,thanosql-dataset/korean_voice_data/audio/broad...,16000,많이 씨앗들을 넣어가지고 너무 촘촘하게 여러 개가 한꺼번에 자라는 거야 여러 줄기가.,많이 씨앗들을 넣어가지고 너무 촘촘하게 여러 개가 한꺼번에 자라는 거야 여러 줄기가.,6.14,I put a lot of seeds in it and it grew into a ...


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" 쿼리 구문을 사용하여 <strong>tutorial_whisper_small</strong> 모델을 예측에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 예측에 사용할 옵션을 지정합니다.
        <ul>
            <li>"audio_col": 예측에 사용할 오디오의 경로가 기록되어 있는 컬럼의 이름 (str, default: 'audio_path')</li>
            <li>"batch_size": 한 번의 학습에 사용하는 데이터 묶음 속의 데이터 수 (int, optional, default: 16)</li>
            <li>"language": 오디오 파일의 주요 사용 언어를 지정. ‘auto’일 경우 모델이 인식할 수 있는 99가지 언어 중 하나로 인식 (str, default: 'auto')</li>
            <li>"task": 실행할 작업의 종류 (str, 'transcribe'|'translate', default: 'transcribe')</li>
            <li>"result_col": 데이터 테이블에서 예측 결과를 담을 컬럼 이름 (str, optional, default: 'predict_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __4. 튜토리얼을 마치며__
 
이번 튜토리얼에서는 korean_voice 데이터 세트를 사용하여 <strong>Whisper</strong>의 기능 중 일반적으로 사용되는 음성 인식과 영어로의 번역을 실행해 보았습니다. 초급 단계 튜토리얼인 만큼 정확도 향상을 위한 과정 설명보다는 작동 위주의 설명으로 진행했습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>음성 인식 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>